# Querying Metadata with the REST API

In this notebook we explore searching for metadata from the REST API. The REST API provides a method to programmatically extract a JSON representation of the meta data from the API.

First we load some python dependancies that we will use as part of this notebook and set the variable `ENDPOINT_URL` to the location of the REST API.

In [1]:
import pandas as pd
import requests
import yaml

with open('config.yml') as file_handle:
    config = yaml.load(file_handle, yaml.FullLoader)

ENDPOINT_URL = config['rest_api']

Below we print the location of the endpoint we will query in this notebook:

In [2]:
print(f"REST API Endpoint: {ENDPOINT_URL}")

REST API Endpoint: http://localhost:8081/json


## Querying Shots with the REST API

We're going to use the python `requests` library to query metadata from get metadata from the database. All we need to do to get a result is to query the database with a HTTP GET at the appropriate endpoint. For example, to get information about different experimental shots we can query the `/json/shots/` endpoint.

In [3]:
response = requests.get(f'{ENDPOINT_URL}/shots')
result = response.json()
print(f"Query returned status code: {response.status_code}")

Query returned status code: 200


The shots endpoint returns a JSON payload with a list of shots. Let's look at the first element from the payload:

In [4]:
result['items'][0]

{'shot_id': 30110,
 'uuid': '48d2b65e-f688-59b5-938f-65af7df0d3af',
 'url': 's3://mast/level1/shots/30110.zarr',
 'timestamp': '2013-09-09T10:51:00',
 'preshot_description': '\nrepeat to get a better density in the 100 -200 ms period - increase flowref from 16.5 to 18.0 in period 90 to 170 ms \n',
 'postshot_description': '\nGood shot - better match to 29994\n',
 'campaign': 'M9',
 'reference_shot': 30108,
 'scenario': 4,
 'heating': 'SS Beam',
 'pellets': False,
 'rmp_coil': False,
 'current_range': '1000 kA',
 'divertor_config': 'Conventional',
 'plasma_shape': 'Connected Double Null',
 'comissioner': None,
 'facility': 'MAST',
 'cpf_p03249': None,
 'cpf_p04673': None,
 'cpf_p04674': None,
 'cpf_p04675': None,
 'cpf_p04676': None,
 'cpf_p04677': None,
 'cpf_p04678': None,
 'cpf_p04679': None,
 'cpf_p04680': None,
 'cpf_p04681': None,
 'cpf_p04833': None,
 'cpf_p04993': None,
 'cpf_p05007': None,
 'cpf_p05008': None,
 'cpf_p05009': None,
 'cpf_p05010': None,
 'cpf_p05011': None,
 'cpf

Each item in the list is a json object. This contains the meta-data items that corresponded to our query. In this case, each item contains information about a different MAST shot. Each item has lots of information about different shots, for example the shot ID, the campaign the shot was part of, the pre- and post-shot description by investigators.

For more information on the what's returned by the API you can look at the endpoint documentation:

https://mast-app.site/redoc


Of course, we can read all this JSON data directly into common python data analysis packages, for example, we can create a `pandas` dataframe directly from the endpoint data

In [5]:
df = pd.read_json(f'{ENDPOINT_URL}/shots')['items']
df = pd.DataFrame(df.to_list())
df.head()

,shot_id,uuid,url,timestamp,preshot_description,postshot_description,campaign,reference_shot,scenario,heating,...,cpf_vol_ipmax,cpf_vol_max,cpf_vol_truby,cpf_wmhd_ipmax,cpf_wmhd_max,cpf_wmhd_truby,cpf_zeff_ipmax,cpf_zeff_max,cpf_zeff_truby,cpf_zmag_efit
0,30110,48d2b65e-f688-59b5-938f-65af7df0d3af,s3://mast/level1/shots/30110.zarr,2013-09-09T10:51:00,\nrepeat to get a better density in the 100 -2...,\nGood shot - better match to 29994\n,M9,30108,4.0,SS Beam,...,8.140033,8.660660,0.0,90911.798669,108315.530,0.0,None,None,None,0.044837
1,30111,f07b1e69-0186-5f5a-b4d8-713242e21b63,s3://mast/level1/shots/30111.zarr,2013-09-09T11:08:00,\nRestore 29764: Enable SW NBI.\n,\ndrop in real time IP caused PCS to shutdown ...,M9,29764,3.0,"2 Beams,SS Beam,SW Beam",...,8.456450,9.741916,0.0,43684.138948,55665.310,0.0,None,None,None,0.042443
2,30113,742c78fa-3811-5404-93b3-5ab419d330bd,s3://mast/level1/shots/30113.zarr,2013-09-09T12:00:00,\nRestore 30111\n,\nShot OK for programme. Two good beams. Big l...,M9,30111,3.0,"2 Beams,SS Beam,SW Beam",...,8.391672,8.426641,0.0,43127.703046,60260.926,0.0,None,None,None,0.042220
3,30116,233c8826-2c14-562f-acf7-09613fc744a8,s3://mast/level1/shots/30116.zarr,2013-09-09T12:49:00,\nRestore 30112: One beam 750kA plasma.\n,\nGood shot\n,M9,30112,4.0,SS Beam,...,7.845821,8.738020,0.0,60943.785535,83505.234,0.0,None,None,None,0.045105
4,30117,d1128db1-a000-505a-8ae6-f4cc4f8c2d06,s3://mast/level1/shots/30117.zarr,2013-09-09T13:05:00,\nRepeat: Increase I(Div) after 300ms.\n,\nGood shot\n,M9,30116,4.0,SS Beam,...,8.683957,8.846392,0.0,58406.060298,84545.930,0.0,None,None,None,0.052367


### Searching & Filtering Data

All REST API endpoints can take query parameters to filter the data returned. For example, we can return all shots for the `M9` campaign by using the approrpiate query string.

For example, we can query for everything from the `M9` campaign by adding `?campaign=M9` to our query string.

In [6]:
df = pd.read_json(f"{ENDPOINT_URL}/shots?campaign=M9")['items']
df = pd.DataFrame(df.to_list())
df.head()

,shot_id,uuid,url,timestamp,preshot_description,postshot_description,campaign,reference_shot,scenario,heating,...,cpf_vol_ipmax,cpf_vol_max,cpf_vol_truby,cpf_wmhd_ipmax,cpf_wmhd_max,cpf_wmhd_truby,cpf_zeff_ipmax,cpf_zeff_max,cpf_zeff_truby,cpf_zmag_efit
0,30110,48d2b65e-f688-59b5-938f-65af7df0d3af,s3://mast/level1/shots/30110.zarr,2013-09-09T10:51:00,\nrepeat to get a better density in the 100 -2...,\nGood shot - better match to 29994\n,M9,30108,4.0,SS Beam,...,8.140033,8.660660,0.0,90911.798669,108315.530,0.0,None,None,None,0.044837
1,30111,f07b1e69-0186-5f5a-b4d8-713242e21b63,s3://mast/level1/shots/30111.zarr,2013-09-09T11:08:00,\nRestore 29764: Enable SW NBI.\n,\ndrop in real time IP caused PCS to shutdown ...,M9,29764,3.0,"2 Beams,SS Beam,SW Beam",...,8.456450,9.741916,0.0,43684.138948,55665.310,0.0,None,None,None,0.042443
2,30113,742c78fa-3811-5404-93b3-5ab419d330bd,s3://mast/level1/shots/30113.zarr,2013-09-09T12:00:00,\nRestore 30111\n,\nShot OK for programme. Two good beams. Big l...,M9,30111,3.0,"2 Beams,SS Beam,SW Beam",...,8.391672,8.426641,0.0,43127.703046,60260.926,0.0,None,None,None,0.042220
3,30116,233c8826-2c14-562f-acf7-09613fc744a8,s3://mast/level1/shots/30116.zarr,2013-09-09T12:49:00,\nRestore 30112: One beam 750kA plasma.\n,\nGood shot\n,M9,30112,4.0,SS Beam,...,7.845821,8.738020,0.0,60943.785535,83505.234,0.0,None,None,None,0.045105
4,30117,d1128db1-a000-505a-8ae6-f4cc4f8c2d06,s3://mast/level1/shots/30117.zarr,2013-09-09T13:05:00,\nRepeat: Increase I(Div) after 300ms.\n,\nGood shot\n,M9,30116,4.0,SS Beam,...,8.683957,8.846392,0.0,58406.060298,84545.930,0.0,None,None,None,0.052367


### Pagination 
The REST API responses are _paginated_, meaning that only a subset of the full items are returned with each query. Pagination is used to limit the total number of requests made by each user to prevent any single user overloading the server with huge data requests.

Pagination information is included in the response and corresponds to [RFC 8288](https://datatracker.ietf.org/doc/html/rfc8288). The response contains cursors at the bottom of the response:

 - `current_page` - this is the cursor relating to the current page of results.
 - `next_page` - this is the cursor for the next page of results.
 - `previous_page` - this is the cursor for the previous page of results, will result in `null` if on the first page.

You can control the pagination using the following options as query arguments:
 - `cursor=xx` to set the cursor for the displayed set of results
 - `size=xx` to set the number of results returned by each page.

In [7]:
response = requests.get(f'{ENDPOINT_URL}/shots?size=2&cursor=Pmk6MzAxMTE%3D')
result = response.json()
headers = response.headers

print("Current page cursor", result['current_page'])
print("Next page cursor", result['next_page'])
print("Previous page cursor", result['previous_page'])

df = pd.DataFrame(result)
df

Current page cursor Pmk6MzAxMTE%3D
Next page cursor Pmk6MzAxMTY%3D
Previous page cursor PGk6MzAxMTM%3D


,items,total,current_page,current_page_backwards,previous_page,next_page
0,"{'shot_id': 30113, 'uuid': '742c78fa-3811-5404...",None,Pmk6MzAxMTE%3D,PGk6MzAxMTc%3D,PGk6MzAxMTM%3D,Pmk6MzAxMTY%3D
1,"{'shot_id': 30116, 'uuid': '233c8826-2c14-562f...",None,Pmk6MzAxMTE%3D,PGk6MzAxMTc%3D,PGk6MzAxMTM%3D,Pmk6MzAxMTY%3D
